In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

This script is for selecting candidates with considerable number of replying users both labeled mute and unmute, for preparing mock tables to insert into datastore.

In [5]:
df = pd.read_csv('/users/sjx/desktop/spec2/mock.csv')
df.head()

,reply_id,reply_user_id,reply_user_name,context_id,replyTo_user,reply_to_name,text,context,from
0,1044937591852863491,696459996603813888,HeatherXoXG,1044698707655094279,3330000369,perrygershon,@perrygershon If you actually listened To @nik...,Our country deserves more than laughter. \nWe ...,tox
1,1050415681332793345,696459996603813888,HeatherXoXG,1050413572927438849,3330000369,perrygershon,@perrygershon @HRC You have got to be kidding ...,We have made great strides down the road towar...,tox
2,1054766195474350080,696459996603813888,HeatherXoXG,1054712699630903301,3330000369,perrygershon,@perrygershon @leezeldin This Ad is Complete C...,Women’s health care is under attack. We see it...,tox
3,1054788551890649089,696459996603813888,HeatherXoXG,1054772685778116613,3330000369,perrygershon,@perrygershon Everything about you is wrong Yo...,This is wrong. \nI know it. You know it. #NY01...,tox
4,1057755057205248000,696459996603813888,HeatherXoXG,1057686202319552514,3330000369,perrygershon,@perrygershon @indyeastend You and the #Democr...,I am proud to be chosen by @indyeastend as the...,tox


In [17]:
stats_from_mock = df[['reply_to_name', 'reply_id', 'from', 'reply_user_id']].\
groupby(['reply_to_name', 'from', 'reply_user_id']).count()\
.reset_index().sort_values(['reply_to_name', 'from', 'reply_id'], ascending=False)

stats_from_mock.to_csv('stats_from_mock.csv')

In [18]:
path = '/Users/sjx/Desktop/SPEC1/data/'
toxic_ready = pd.read_csv(path+'all_toxic_reply_ready.csv')
nontoxic_ready = pd.read_csv(path+'all_nontoxic_reply_ready.csv')
toxic_ready['from'] = 'tox'
nontoxic_ready['from'] = 'nontox'
ready = pd.concat([toxic_ready, nontoxic_ready], ignore_index=True)
ready.shape

(108058, 16)

In [19]:
ready.columns

Index(['Unnamed: 0', 'tweet_id', 'user_id', 'user_handle', 'user_party',
       'mentions', 'replyTo', 'replyTo_user', 'candidate_name',
       'candidate_handle', 'candidate_party', 'text', 'urls', 'context',
       'context_urls', 'from'],
      dtype='object')

In [20]:
ready.drop(['Unnamed: 0',  'user_party', 'mentions', 'candidate_party', 'urls', 'context_urls'], inplace=True, axis=1)
ready.head()

,tweet_id,user_id,user_handle,replyTo,replyTo_user,candidate_name,candidate_handle,text,context,from
0,1045112299126632450,866458494752497664,bren2915,1045072954109956097,18166778,Jim Jordan (Ohio),Jim_Jordan,@Jim_Jordan YAY!!! You are out of step with A...,H.R. 6157 (the CROmnibus) just passed. I voted...,tox
1,1045079408778911745,3816874095,KevinWa22989694,1045072954109956097,18166778,Jim Jordan (Ohio),Jim_Jordan,@Jim_Jordan You guys are doing a horrible job ...,H.R. 6157 (the CROmnibus) just passed. I voted...,tox
2,1045131369150656513,870551766,AChrisOstler,1045072954109956097,18166778,Jim Jordan (Ohio),Jim_Jordan,@Jim_Jordan Congress sucks so bad! The corrupt...,H.R. 6157 (the CROmnibus) just passed. I voted...,tox
3,1045169156641812480,773748515956748288,Maryannepaulso3,1045072954109956097,18166778,Jim Jordan (Ohio),Jim_Jordan,@Jim_Jordan Hey Jim have you apologised to you...,H.R. 6157 (the CROmnibus) just passed. I voted...,tox
4,1045094012267286528,274679317,scotapathy,1045072954109956097,18166778,Jim Jordan (Ohio),Jim_Jordan,@Jim_Jordan Ah. You prioritize a useless wall ...,H.R. 6157 (the CROmnibus) just passed. I voted...,tox


In [ ]:
stats_ready = ready[['tweet_id', 'candidate_name', 'from']].groupby(['candidate_name', 'from'])\
.count().reset_index().sort_values('tweet_id', ascending=False)
# stats_ready.to_csv('stats_ready.csv')

In [30]:
top_tox_cand = stats_ready[stats_ready['from']=='tox'].sort_values('tweet_id', ascending=False)
top_tox_cand_list = top_tox_cand.loc[top_tox_cand['tweet_id'] > 200, 'candidate_name'].tolist()

In [31]:
top_nontox_cand = stats_ready[stats_ready['from']=='nontox'].sort_values('tweet_id', ascending=False)
top_nontox_cand_list = top_nontox_cand.loc[top_nontox_cand['tweet_id'] > 200, 'candidate_name'].tolist()

In [35]:
top_cand_list = list(set(top_nontox_cand_list).intersection(set(top_tox_cand_list)))
top_cand_list

['Adam Schiff',
 'Kevin McCarthy (California)',
 'Jason Lewis (Minnesota)',
 'Joseph Kennedy III',
 'Cathy McMorris Rodgers',
 'Janet Garrett',
 'Claudia Tenney',
 'Pramila Jayapal',
 'Tom MacArthur',
 'Matt Gaetz',
 'Alexandria Ocasio-Cortez',
 'Lee Zeldin',
 'Randy Bryce',
 'Mark Meadows (North Carolina)',
 'Nancy Pelosi',
 'Ralph Norman',
 'Daniel Crenshaw',
 'Joaquin Castro',
 'Ted Lieu',
 'Eric Swalwell',
 'John Faso',
 'Sheila Jackson Lee',
 'Steve Scalise']

In [43]:
top_ready = ready[ready['candidate_name'].isin(top_cand_list)].copy()
stats_from_top_ready = top_ready[['tweet_id', 'candidate_name', 'user_id', 'from']].groupby(['candidate_name', 'user_id', 'from'])\
.count().reset_index().sort_values(['candidate_name', 'user_id'])
stats_from_top_ready.to_csv('stats_from_top_ready.csv')

In [44]:
len(top_ready['candidate_name'].unique())

23

In [47]:
# try
adams = stats_from_top_ready[stats_from_top_ready['candidate_name']=='Adam Schiff'].copy()
binary_users = set()
for i, row in adams.iterrows():
    if row['user_id'] not in binary_users:
        binary_users.add(row['user_id'])
    else:
        binary_users.remove(row['user_id'])
print(len(binary_users), len(adams))
print(adams[~adams['user_id'].isin(binary_users)].index)
adams.drop(adams[~adams['user_id'].isin(binary_users)].index, inplace=True)
adams.shape

568 576
Int64Index([157, 158, 287, 288, 388, 389, 549, 550], dtype='int64')


(568, 4)

In [49]:
top_cand_info_list = []
for cand in top_cand_list:
    adams = stats_from_top_ready[stats_from_top_ready['candidate_name']==cand].copy()
    binary_users = set()
    for i, row in adams.iterrows():
        if row['user_id'] not in binary_users:
            binary_users.add(row['user_id'])
        else:
            binary_users.remove(row['user_id'])
#     print(adams[~adams['user_id'].isin(binary_users)].index)
    print(cand, len(binary_users), len(adams), )
    adams.drop(adams[~adams['user_id'].isin(binary_users)].index, inplace=True)
    top_cand_info_list.append(adams)
print(len(top_cand_info_list))

Adam Schiff 568 576
Kevin McCarthy (California) 397 407
Jason Lewis (Minnesota) 233 313
Joseph Kennedy III 535 545
Cathy McMorris Rodgers 308 418
Janet Garrett 416 436
Claudia Tenney 234 326
Pramila Jayapal 486 530
Tom MacArthur 451 515
Matt Gaetz 527 533
Alexandria Ocasio-Cortez 629 643
Lee Zeldin 397 475
Randy Bryce 505 517
Mark Meadows (North Carolina) 633 667
Nancy Pelosi 543 557
Ralph Norman 657 687
Daniel Crenshaw 827 831
Joaquin Castro 645 679
Ted Lieu 502 510
Eric Swalwell 672 680
John Faso 257 355
Sheila Jackson Lee 580 638
Steve Scalise 638 644
23


In [52]:
final_cand_df_list = []
for info in top_cand_info_list:
    uids = set(info['user_id'].tolist())
    cand = list(set(info['candidate_name'].tolist()))[0]
    print(cand)
    cand_df = ready[(ready['candidate_name']==cand) & (ready['user_id'].isin(uids))].copy()
    final_cand_df_list.append(cand_df)
print(len(final_cand_df_list))
final_cand_df_list[0].head()

Adam Schiff
Kevin McCarthy (California)
Jason Lewis (Minnesota)
Joseph Kennedy III
Cathy McMorris Rodgers
Janet Garrett
Claudia Tenney
Pramila Jayapal
Tom MacArthur
Matt Gaetz
Alexandria Ocasio-Cortez
Lee Zeldin
Randy Bryce
Mark Meadows (North Carolina)
Nancy Pelosi
Ralph Norman
Daniel Crenshaw
Joaquin Castro
Ted Lieu
Eric Swalwell
John Faso
Sheila Jackson Lee
Steve Scalise
23


,tweet_id,user_id,user_handle,replyTo,replyTo_user,candidate_name,candidate_handle,text,context,from
104,1042189802089709568,61192537,Makearight,1041841277074132993,29501253,Adam Schiff,RepAdamSchiff,@RepAdamSchiff Your pending investigation is a...,President Trump has intervened again in a pend...,tox
564,1051902157908250626,1576197966,calimom61,1051856388199071744,29501253,Adam Schiff,RepAdamSchiff,@RepAdamSchiff How dare you put Kavanaugh in t...,King Salman denies it.\n\nVladimir Putin denie...,tox
682,1042065390027853824,3346877764,mateoqme,1041841277074132993,29501253,Adam Schiff,RepAdamSchiff,@RepAdamSchiff Schiff seriously shut the hell ...,President Trump has intervened again in a pend...,tox
696,1042646037058314240,965372755024609281,SherlockeHomey,1041841277074132993,29501253,Adam Schiff,RepAdamSchiff,@RepAdamSchiff THE STANDARD HOTEL CROSSED A RE...,President Trump has intervened again in a pend...,tox
795,1042220659219292160,975770554081988610,EarnestDenaro,1041841277074132993,29501253,Adam Schiff,RepAdamSchiff,"@RepAdamSchiff Eat sh*t, you communist twink b...",President Trump has intervened again in a pend...,tox


In [54]:
fields = ['FLIRTATION','IDENTITY_ATTACK','INFLAMMATORY','INSULT','OBSCENE','PROFANITY',
          'SEVERE_TOXICITY','SEXUALLY_EXPLICIT','THREAT','TOXICITY','id']
processed = pd.read_csv('/Users/sjx/Desktop/SPEC1/data/all/tweetsData_preprocessed.csv', 
                        skipinitialspace=True, usecols=fields)
processed.shape

(1564293, 11)

In [56]:
adams = final_cand_df_list[0]
print(adams.shape)
new_adams = pd.merge(adams, processed, how='inner', left_on='tweet_id', right_on='id')
print(new_adams.shape)
print(new_adams.columns)

(575, 10)
(575, 21)
Index(['tweet_id', 'user_id', 'user_handle', 'replyTo', 'replyTo_user',
       'candidate_name', 'candidate_handle', 'text', 'context', 'from',
       'FLIRTATION', 'IDENTITY_ATTACK', 'INFLAMMATORY', 'INSULT', 'OBSCENE',
       'PROFANITY', 'SEVERE_TOXICITY', 'SEXUALLY_EXPLICIT', 'THREAT',
       'TOXICITY', 'id'],
      dtype='object')


In [57]:
final_tox_df_list = []
for df in final_cand_df_list:
    new_df = pd.merge(df, processed, how='inner', left_on='tweet_id', right_on='id')
    new_df.drop('id', axis=1, inplace=True)
    print(df.shape, new_df.shape)
    final_tox_df_list.append(new_df)
print(len(final_tox_df_list))
final_tox_df_list[0].head()

(575, 10) (575, 20)
(422, 10) (422, 20)
(399, 10) (399, 20)
(547, 10) (547, 20)
(398, 10) (398, 20)
(437, 10) (437, 20)
(345, 10) (345, 20)
(529, 10) (529, 20)
(521, 10) (521, 20)
(551, 10) (551, 20)
(640, 10) (640, 20)
(493, 10) (493, 20)
(533, 10) (533, 20)
(680, 10) (680, 20)
(559, 10) (559, 20)
(686, 10) (686, 20)
(846, 10) (846, 20)
(673, 10) (673, 20)
(521, 10) (521, 20)
(680, 10) (680, 20)
(335, 10) (335, 20)
(629, 10) (629, 20)
(648, 10) (648, 20)
23


,tweet_id,user_id,user_handle,replyTo,replyTo_user,candidate_name,candidate_handle,text,context,from,FLIRTATION,IDENTITY_ATTACK,INFLAMMATORY,INSULT,OBSCENE,PROFANITY,SEVERE_TOXICITY,SEXUALLY_EXPLICIT,THREAT,TOXICITY
0,1042189802089709568,61192537,Makearight,1041841277074132993,29501253,Adam Schiff,RepAdamSchiff,@RepAdamSchiff Your pending investigation is a...,President Trump has intervened again in a pend...,tox,0.398202,0.316839,0.453808,0.825058,0.406679,0.648483,0.559063,0.229256,0.386605,0.768174
1,1051902157908250626,1576197966,calimom61,1051856388199071744,29501253,Adam Schiff,RepAdamSchiff,@RepAdamSchiff How dare you put Kavanaugh in t...,King Salman denies it.\n\nVladimir Putin denie...,tox,0.366911,0.292537,0.346792,0.767115,0.217947,0.588425,0.492098,0.187887,0.311471,0.765579
2,1042065390027853824,3346877764,mateoqme,1041841277074132993,29501253,Adam Schiff,RepAdamSchiff,@RepAdamSchiff Schiff seriously shut the hell ...,President Trump has intervened again in a pend...,tox,0.343481,0.329185,0.663279,0.797087,0.955556,0.667156,0.616942,0.211560,0.472687,0.845272
3,1042646037058314240,965372755024609281,SherlockeHomey,1041841277074132993,29501253,Adam Schiff,RepAdamSchiff,@RepAdamSchiff THE STANDARD HOTEL CROSSED A RE...,President Trump has intervened again in a pend...,tox,0.602242,0.554676,0.807930,0.727237,0.126603,0.712436,0.701178,0.868834,0.638913,0.738868
4,1042220659219292160,975770554081988610,EarnestDenaro,1041841277074132993,29501253,Adam Schiff,RepAdamSchiff,"@RepAdamSchiff Eat sh*t, you communist twink b...",President Trump has intervened again in a pend...,tox,0.599214,0.636659,0.630072,0.773049,0.988132,0.843233,0.725708,0.652246,0.655133,0.802834


In [59]:
for df in final_tox_df_list:
    cand = list(set(df['candidate_name'].tolist()))[0]    
    df.to_csv(cand+'.csv', index=False)

In [73]:
df_shape, tox_shape, nontox_shape = [], [], []
for df in final_tox_df_list:
    cand = list(set(df['candidate_name'].tolist()))[0]
    print(cand, df.shape[0], df[df['from']=='tox'].shape[0], df[df['from']=='nontox'].shape[0])
    df_shape.append(df.shape[0]) 
    tox_shape.append(df[df['from']=='tox'].shape[0]) 
    nontox_shape.append(df[df['from']=='nontox'].shape[0])

stats = pd.DataFrame({'cand':top_cand_list, 
                     'total_users':df_shape, 
                     'mute':tox_shape, 
                     'nonmute':nontox_shape})
print(stats.mean())
stats

Adam Schiff 575 302 273
Kevin McCarthy (California) 422 195 227
Jason Lewis (Minnesota) 399 102 297
Joseph Kennedy III 547 303 244
Cathy McMorris Rodgers 398 185 213
Janet Garrett 437 248 189
Claudia Tenney 345 115 230
Pramila Jayapal 529 239 290
Tom MacArthur 521 240 281
Matt Gaetz 551 294 257
Alexandria Ocasio-Cortez 640 356 284
Lee Zeldin 493 189 304
Randy Bryce 533 267 266
Mark Meadows (North Carolina) 680 372 308
Nancy Pelosi 559 281 278
Ralph Norman 686 341 345
Daniel Crenshaw 846 498 348
Joaquin Castro 673 345 328
Ted Lieu 521 277 244
Eric Swalwell 680 382 298
John Faso 335 105 230
Sheila Jackson Lee 629 325 304
Steve Scalise 648 305 343
total_users    549.869565
mute           272.434783
nonmute        277.434783
dtype: float64


,cand,total_users,mute,nonmute
0,Adam Schiff,575,302,273
1,Kevin McCarthy (California),422,195,227
2,Jason Lewis (Minnesota),399,102,297
3,Joseph Kennedy III,547,303,244
4,Cathy McMorris Rodgers,398,185,213
5,Janet Garrett,437,248,189
6,Claudia Tenney,345,115,230
7,Pramila Jayapal,529,239,290
8,Tom MacArthur,521,240,281
9,Matt Gaetz,551,294,257


#### upload to datastore

In [61]:
path = '/Users/sjx/Desktop/SPEC2/upload/'
dc = pd.read_csv(path+'DanielCrenshaw.csv')
aoc = pd.read_csv(path+'AlexandriaOcasioCortez.csv')
mm = pd.read_csv(path+'MarkMeadows.csv')
ads = pd.read_csv(path+'AdamSchiff.csv')
np = pd.read_csv(path+'NancyPelosi.csv')
cands = [dc, aoc, mm, ads, np]

In [64]:
for c in cands:
    print(c.shape, c[c['from']=='tox'].shape, c[c['from']=='nontox'].shape)

(846, 20) (498, 20) (348, 20)
(640, 20) (356, 20) (284, 20)
(680, 20) (372, 20) (308, 20)
(575, 20) (302, 20) (273, 20)
(559, 20) (281, 20) (278, 20)


In [68]:
dc1 = dc.sample(frac=0.5).copy()
dc2 = dc.loc[~dc['tweet_id'].isin(dc1['tweet_id'].tolist())]
print(dc1.shape, dc2.shape)
dc1.to_csv('dc1.csv', index=False)
dc2.to_csv('dc2.csv', index=False)

(423, 20) (423, 20)


In [69]:
dc1.columns

Index(['tweet_id', 'user_id', 'user_handle', 'replyTo', 'replyTo_user',
       'candidate_name', 'candidate_handle', 'text', 'context', 'from',
       'FLIRTATION', 'IDENTITY_ATTACK', 'INFLAMMATORY', 'INSULT', 'OBSCENE',
       'PROFANITY', 'SEVERE_TOXICITY', 'SEXUALLY_EXPLICIT', 'THREAT',
       'TOXICITY'],
      dtype='object')